<a href="https://colab.research.google.com/github/ayush24578/Concepts-and-Technologies-of-AI/blob/main/Ayush_Dhungana_Worksheet8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [9]:
import numpy as np

class CustomDecisionTree:
  def __init__(self, max_depth=None):
    self.max_depth = max_depth
    self.tree = None

  def fit(self, X, y):
    self.tree = self._build_tree(X, y)

  def _build_tree(self, X, y, depth=0):
    num_samples, num_features = X.shape
    unique_classes = np.unique(y)

    # Stopping conditions: pure node or reached max depth
    if len(unique_classes) == 1:
      return {'class': unique_classes[0]}

    if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
      return {'class': np.bincount(y).argmax()}

    best_info_gain = -float('inf')
    best_split = None

    for feature_idx in range(num_features):
      thresholds = np.unique(X[:, feature_idx])

      for threshold in thresholds:
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask

        left_y = y[left_mask]
        right_y = y[right_mask]

        if len(left_y) == 0 or len(right_y) == 0:
          continue

        info_gain = self._information_gain(y, left_y, right_y)

        if info_gain > best_info_gain:
          best_info_gain = info_gain
          best_split = {
            'feature_idx': feature_idx,
            'threshold': threshold,
            'left_mask': left_mask,
            'right_mask': right_mask
          }

    if best_split is None:
      return {'class': np.bincount(y).argmax()}

    left_tree = self._build_tree(
      X[best_split['left_mask']],
      y[best_split['left_mask']],
      depth + 1
    )

    right_tree = self._build_tree(
      X[best_split['right_mask']],
      y[best_split['right_mask']],
      depth + 1
    )

    return {
      'feature_idx': best_split['feature_idx'],
      'threshold': best_split['threshold'],
      'left_tree': left_tree,
      'right_tree': right_tree
    }

  def _information_gain(self, parent, left, right):
    parent_entropy = self._entropy(parent)
    left_entropy = self._entropy(left)
    right_entropy = self._entropy(right)

    weighted_avg_entropy = (
      (len(left) / len(parent)) * left_entropy +
      (len(right) / len(parent)) * right_entropy
    )

    return parent_entropy - weighted_avg_entropy

  def _entropy(self, y):
    if len(y) == 0:
      return 0
    class_probs = np.bincount(y) / len(y)
    return -np.sum(class_probs * np.log2(class_probs + 1e-9))

  def predict(self, X):
    return [self._predict_single(x, self.tree) for x in X]

  def _predict_single(self, x, tree):
    if 'class' in tree:
      return tree['class']

    feature_val = x[tree['feature_idx']]

    if feature_val <= tree['threshold']:
      return self._predict_single(x, tree['left_tree'])
    else:
      return self._predict_single(x, tree['right_tree'])


In [10]:
data = load_iris()
X = data.data
y = data.target
# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 1.0000


In [12]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


In [13]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


In [14]:
# Import necessary libraries
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import f1_score, mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load Wine dataset
wine_data = load_wine()
X_wine = wine_data.data
y_wine = wine_data.target


X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_wine, y_wine, test_size=0.2, random_state=42
)

dt_classifier = DecisionTreeClassifier(random_state=42)
rf_classifier = RandomForestClassifier(random_state=42)

dt_classifier.fit(X_train_w, y_train_w)
rf_classifier.fit(X_train_w, y_train_w)


y_pred_dt = dt_classifier.predict(X_test_w)
y_pred_rf = rf_classifier.predict(X_test_w)

f1_dt = f1_score(y_test_w, y_pred_dt, average='weighted')
f1_rf = f1_score(y_test_w, y_pred_rf, average='weighted')

print("=== Classification Results ===")
print(f"Decision Tree F1 Score: {f1_dt:.4f}")
print(f"Random Forest F1 Score: {f1_rf:.4f}")

=== Classification Results ===
Decision Tree F1 Score: 0.9440
Random Forest F1 Score: 1.0000


In [15]:
# 2. Hyperparameter Tuning with GridSearchCV

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

grid_search.fit(X_train_w, y_train_w)

# Results
print("\n=== GridSearchCV Results ===")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation F1 Score: {grid_search.best_score_:.4f}")

best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test_w)
f1_best = f1_score(y_test_w, y_pred_best, average='weighted')
print(f"Test Set F1 Score with Best Model: {f1_best:.4f}")



=== GridSearchCV Results ===
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation F1 Score: 0.9783
Test Set F1 Score with Best Model: 1.0000


In [16]:
# Load regression dataset
from sklearn.datasets import load_diabetes

# Load diabetes dataset for regression
diabetes = load_diabetes()
X_reg = diabetes.data
y_reg = diabetes.target

# Split into train and test sets
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# 3. Train Decision Tree and Random Forest Regressors
dt_regressor = DecisionTreeRegressor(random_state=42)
rf_regressor = RandomForestRegressor(random_state=42)

# Fit models
dt_regressor.fit(X_train_r, y_train_r)
rf_regressor.fit(X_train_r, y_train_r)


y_pred_dt_r = dt_regressor.predict(X_test_r)
y_pred_rf_r = rf_regressor.predict(X_test_r)

mse_dt = mean_squared_error(y_test_r, y_pred_dt_r)
mse_rf = mean_squared_error(y_test_r, y_pred_rf_r)

print("\n=== Regression Results ===")
print(f"Decision Tree Regressor MSE: {mse_dt:.4f}")
print(f"Random Forest Regressor MSE: {mse_rf:.4f}")

# 4. Hyperparameter Tuning for Random Forest Regressor (RandomizedSearchCV)
from sklearn.model_selection import RandomizedSearchCV

param_dist_rf_reg = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist_rf_reg,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_r, y_train_r)

# Results
print("\n=== RandomizedSearchCV Results ===")
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Cross-Validation MSE: {-random_search.best_score_:.4f}")


best_rf_reg = random_search.best_estimator_
y_pred_best_reg = best_rf_reg.predict(X_test_r)
mse_best = mean_squared_error(y_test_r, y_pred_best_reg)
print(f"Test Set MSE with Best Model: {mse_best:.4f}")


=== Regression Results ===
Decision Tree Regressor MSE: 4976.7978
Random Forest Regressor MSE: 2952.0106

=== RandomizedSearchCV Results ===
Best Parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10}
Best Cross-Validation MSE: 3318.2050
Test Set MSE with Best Model: 2828.4947
